In [1]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestRegressor as RFR

In [2]:
def get_medians(train_file):
    user_plays = {}
    artist_plays = {}
    triples = []
    plays = []
    with open(train_file, 'rb') as train_fh:
        train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
        next(train_csv, None)
        for row in train_csv:
            user = row[0]
            artist = row[1]
            plays = int(row[2])

            if user not in user_plays:
                user_plays[user] = []
            user_plays[user].append(plays)

            if artist not in artist_plays:
                artist_plays[artist] = []
            artist_plays[artist].append(plays)

            triples.append((user, artist, plays))


    artist_medians = {}
    for artist in artist_plays:
        artist_medians[artist] = np.mean(artist_plays[artist])

    user_medians = {}
    for user in user_plays:
        user_medians[user] = np.mean(user_plays[user])

    X = []
    Y = []
    for user, artist, play in triples:
        X.append((user_medians[user], artist_medians[artist]))
        Y.append(play)

    global_median = np.mean(Y)

    return X, Y, global_median, user_medians, artist_medians

In [3]:
X, Y, global_median, user_medians, artist_medians = get_medians("train.csv")

In [6]:
global_median = float(global_median)
exp = 0.3
diff = 0.01
count = 0

while count < 40:
    predictions = []
    phi_X = []
    for user_med, artist_med in X:
        predictions.append(user_med * (artist_med / global_median) ** exp)

    n = float(len(Y))
    error = 0
    for i in range(int(n)):
        error += 1.0 / n * abs(Y[i] - predictions[i])

    print exp, error
    
    exp += diff
    count += 1

0.3 150.688551895
0.31 150.67050854
0.32 150.655691657
0.33 150.64404575
0.34 150.635571164
0.35 150.630342373
0.36 150.628392647
0.37 150.629675694
0.38 150.634087382
0.39 150.641718895
0.4 150.652549862
0.41 150.66663734
0.42 150.683977943


KeyboardInterrupt: 

In [ ]:
predictions = np.asarray(predictions).reshape(-1,1)
model = RFR()
model.fit(predictions, Y)

In [ ]:
model_predicts = model.predict(predictions)

In [ ]:
error = 0
n = len(model_predicts)
for i in range(int(n)):
    error += 1.0 / n * abs(Y[i] - model_predicts[i])
print error

In [ ]:
with open('end_of_test.csv', 'rb') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    next(test_csv, None)
    with open("simple_preds_RF_10_end.csv", 'w') as sp_fh:
        sp_csv = csv.writer(sp_fh, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        sp_csv.writerow(['Id', 'plays'])
        count = 0
        for row in test_csv:
            user = row[1]
            artist = row[2]
            prediction = model.predict(user_medians[user] * (artist_medians[artist] / global_median) ** exp)[0]
            sp_csv.writerow([count, round(prediction,6)])
            count += 1